# Valimp tutorial

## Index

- [Introduction](#Introduction)
- [Type validation](#Type-validation)
    - [Built-in and custom types](#Built-in-and-custom-types)
    - [Methods](#Methods)
    - [Type unions and optional types (the `|` operator)](#Type-unions-and-optional-types-(the-|-operator))
    - [Validation of container items](#Validation-of-container-items)
        - [`list`](#list-and-collections.abc.Sequence) and [`collections.abc.Sequence`](#list-and-collections.abc.Sequence)
        - [`dict`](#dict-and-collections.abc.Mapping) and [`collections.abc.Mapping`](#dict-and-collections.abc.Mapping)
        - [`tuple`](#tuple)
        - [`set`](#set)
        - [`valimp.NO_ITEM_VALIDATION`](#valimp.NO_ITEM_VALIDATION)
        - [Nested containers](#Nested-containers)
    - [`collections.abc.Callable`](#collections.abc.Callable)
    - [`typing.Literal`](#typing.Literal)
- [Signature validation](#Signature-validation)
- [Coerce](#Coerce)
    - [Type checkers](#Type-checkers)
- [Parsing](#Parsing)
    - [Custom validation](#Custom-validation)
    - [Dynamic default values](#Dynamic-default-values)
    - [parse_none](#parse_none)
- [Coerce and parse](#Coerce-and-parse)

## Notes

### Error messages
Tracebacks have been curtailed in some displayed error messages. To see a full exception as raised just execute the corresponding cells.

## Introduction

Valimp uses type annotations (hints) to easily validate, parse and coerce inputs to public functions and methods.

Adding the `valimp.parse` decorator to a 'type-annotated' function or method will:
  - validate all inputs against the type annotation, including optional type validation of items in containers.
  - validate inputs against the function signature.
  - provide for coercing inputs to a specific type.
  - provide for user-defined parsing and custom validation.

## Type validation

### Built-in and custom types

Just add the `@valimp.parse` decorator to a type-annotated function. That's it!

In [1]:
from valimp import parse
from typing import Any

class MyCustomType(int):
    """I don't add much."""

@parse
def public_function(
    a: str,
    b: int,
    c: list,
    *,
    kw_a: bool,
    kw_b: MyCustomType,
):
    return

all the following inputs are valid...

In [2]:
rtrn = public_function(
    "a string",
    1,
    [2],
    kw_a=True,
    kw_b=MyCustomType(4)
)
assert rtrn is None

If at least one input is not valid then a `valimp.InputsError` is raised advising of each parameter for which an input does not conform with the corresponding type annotation.

In [ ]:
public_function(
    a=["not a string", "but a list"],  # INVALID, not a str
    b="not an int",  # INVALID, not a str
    c=2,  # INVALID, not a list
    kw_a=None,  # INVALID, None not an option
    kw_b=True,  # INVALID, not the required custom type
)

```
---------------------------------------------------------------------------
InputsError                               Traceback (most recent call last)
Cell In[3], line 1
----> 1 public_function(

InputsError: The following inputs to 'public_function' do not conform with the corresponding type annotation:

a
	Takes type <class 'str'> although received '['not a string', 'but a list']' of type <class 'list'>.

b
	Takes type <class 'int'> although received 'not an int' of type <class 'str'>.

c
	Takes type <class 'list'> although received '2' of type <class 'int'>.

kw_a
	Takes type <class 'bool'> although received 'None' of type <class 'NoneType'>.

kw_b
	Takes type <class '__main__.MyCustomType'> although received 'True' of type <class 'bool'>.
```

### Methods

Works just the same for a method:

In [4]:
class SomeClass:
    
    @parse
    def public_method(
        self,
        a: str,
        *,
        kw_a: MyCustomType,
    ):
        return

sc = SomeClass()
# valid inputs...
assert sc.public_method("a string", kw_a=MyCustomType(4)) is None

In [ ]:
sc.public_method(0, kw_a=None)

```
---------------------------------------------------------------------------
InputsError                               Traceback (most recent call last)
Cell In[5], line 1
----> 1 sc.public_method(0, kw_a=None)

InputsError: The following inputs to 'public_method' do not conform with the corresponding type annotation:

a
	Takes type <class 'str'> although received '0' of type <class 'int'>.

kw_a
	Takes type <class '__main__.MyCustomType'> although received 'None' of type <class 'NoneType'>.
```

Henceforth all examples in this tutorial will be defined as functions, although everything works in the same way for methods.

### Type unions and optional types (the `|` operator)

Valimp supports annotations for type unions and optional types.

From python 3.9 the `typing.Union` and `typing.Optional` annotations can be used to define type hints. From python 3.10 the alternative `|` operator can be used.

In [6]:
# for python >=3.9
from typing import Union, Optional

@parse
def pf(
    a: Union[int, float],  
    b: Union[int, float, None],  # NB can include None directly within Union
    c: Union[int, float, None],
    d: Optional[int],
    e: Optional[int] = None,
):
    return

In [ ]:
# for python >=3.10
@parse
def pf_310(
    a: int | float,  
    b: int | float | None,  # NB can include None directly within Union
    c: int | float | None,
    d: int | None,
    e: int | None = None,
):
    return

**NOTE** All other examples in this tutorial that provide for type unions or optional typing use the `typing.Union` and `typing.Optional` classes. If using python >=3.10 then the `|` operator could alternatively be used.

In [7]:
assert pf(0, 1.1, None, 3, None) is None # valid inputs

In [ ]:
pf(
    a=None,  # INVALID as None not an option in the Union
    b="not valid",  # INVALID as not an int, float or None
    c=None,  # valid
    d="not_valid",  # INVALID as not an int or None
  )

```
---------------------------------------------------------------------------
InputsError                               Traceback (most recent call last)
Cell In[8], line 1
----> 1 pf(

InputsError: The following inputs to 'pf' do not conform with the corresponding type annotation:

a
	Takes input that conforms with <(<class 'int'>, <class 'float'>)> although received 'None' of type <class 'NoneType'>.

b
	Takes input that conforms with <(<class 'int'>, <class 'float'>, <class 'NoneType'>)> although received 'not valid' of type <class 'str'>.

d
	Takes input that conforms with <(<class 'int'>, <class 'NoneType'>)> although received 'not_valid' of type <class 'str'>.
```

### Validation of container items

By default Valimp will validate that the items of a container conform with any type subscriptions. For example an input to a parameter annotated as `param: list[int]` will be validated as a list and all items containined in that list will be validated as being of type `int`.

The [valimp.NO_ITEM_VALIDATION](#valimp.NO_ITEM_VALIDATION) section shows how to **not** validate the type of contained items.

#### `list` and `collections.abc.Sequence`
`list` and `collections.abc.Sequence` can both be subscripted with a single argument defining the annotation that all contained items should conform to.

In [9]:
from collections.abc import Sequence

@parse
def pf(
    a: list[int],
    b: Sequence[str],
    c: list[Union[int, float]],
    d: list[Union[int, float]],
    e: Sequence[Optional[int]],
):
    return

In [10]:
rtrn = pf(  # all valid inputs
    a=[0, 1, 2],
    b=["b", "bb", "bbb"],
    c=[2, 2.1, 2.2],
    d=[3, 3, 3.3],
    e=(4, None, 5, None),
)
assert rtrn is None

In [ ]:
pf(
    a=[0, "1", 2],  # INVALID, contains a str
    b=["b", 0, "bbb"],  # INVALID, contains an int
    c=[2, "2.1", 2.2],  # INVALID, contains a str
    d=[3, None, 3.3],  # INVALID, contains None
    e=(4, None, 5.0, None),  # INVALID, contains a float
)

```
---------------------------------------------------------------------------
InputsError                               Traceback (most recent call last)
Cell In[11], line 1
----> 1 pf(

InputsError: The following inputs to 'pf' do not conform with the corresponding type annotation:

a
	Takes type <class 'list'> containing items that conform with <list[int]>, although the received container contains item '1' of type <class 'str'>.

b
	Takes type <class 'collections.abc.Sequence'> containing items that conform with <collections.abc.Sequence[str]>, although the received container contains item '0' of type <class 'int'>.

c
	Takes type <class 'list'> containing items that conform with <list[typing.Union[int, float]]>, although the received container contains item '2.1' of type <class 'str'>.

d
	Takes type <class 'list'> containing items that conform with <list[typing.Union[int, float]]>, although the received container contains item 'None' of type <class 'NoneType'>.

e
	Takes type <class 'collections.abc.Sequence'> containing items that conform with <collections.abc.Sequence[typing.Optional[int]]>, although the received container contains item '5.0' of type <class 'float'>.
```

#### `dict` and `collections.abc.Mapping`

Valimp will validate both the keys and values of inputs annotated with `dict` or `collections.abc.Mapping`.

NB Annotations for these classes take two subscriptions, the first representing the keys and the second representing the values.

In [12]:
from collections.abc import Mapping

@parse
def pf(
    a: dict[str, int],
    b: dict[str, int],
    c: dict[str, int],
    d: Mapping[str, Union[int, float]],
    e: dict[str, Optional[str]],
):
    return

In [13]:
rtrn = pf(  # valid inputs 
    a={"a": 0, "aa": 0},
    b={"b": 1, "bb": 1},
    c={"c": 2, "cc": 2},
    d={"d": 3, "dd": 3.3},
    e={"e": "four", "ee": None},
)
assert rtrn is None

In [ ]:
pf(
    a={0: 0, "aa": 1},  # INVALID, has a key as an int
    b={"b": 1, "bb": "one"},  # INVALID, has a value as a str
    c={2: "two", "cc": 2},  # INVALID, has a key as int and value as str
    d={"d": 3, "dd": None},  # INVALID, has a value as None
    e={"e": 4, "ee": None},  # INVALID, has a value as an int
)

```
---------------------------------------------------------------------------
InputsError                               Traceback (most recent call last)
Cell In[14], line 1
----> 1 pf(

InputsError: The following inputs to 'pf' do not conform with the corresponding type annotation:

a
	Takes type <class 'dict'> with keys that conform to the first argument of <dict[str, int]>, although the received dictionary contains key '0' of type <class 'int'>.

b
	Takes type <class 'dict'> with values that conform to the second argument of <dict[str, int]>, although the received dictionary contains value 'one' of type <class 'str'>.

c
	Takes type <class 'dict'> with keys that conform to the first argument and values that conform to the second argument of <dict[str, int]>, although the received dictionary contains an item with key '2' of type <class 'int'> and value 'two' of type <class 'str'>.

d
	Takes type <class 'collections.abc.Mapping'> with values that conform to the second argument of <collections.abc.Mapping[str, typing.Union[int, float]]>, although the received mapping contains value 'None' of type <class 'NoneType'>.

e
	Takes type <class 'dict'> with values that conform to the second argument of <dict[str, typing.Optional[str]]>, although the received dictionary contains value '4' of type <class 'int'>.
```

#### `tuple`

Python provides two ways to annotate a tuple, both of which are supported by Valimp.
* To declare that a tuple **behaves like a generic sequence**, of arbitrary length with all items conforming to the same annotation, the tuple annotation takes two arguments. The first argument defines the annotation to which all items should conform. The second takes `Ellipsis`. For example, `tuple[Union[int, float], ...]` declares that an input takes an arbitrary length tuple containing objects of either `int` or `float`.
* To declare a tuple of **specific length** the annotation takes the same number of arguments as the tuple's length. Each argument takes the annotation that the item at the corresponding position should conform to. For example, `tuple[int, str, Optional[str]]` declares a 3-tuple which must contain an `int` at index 0, a `str` at index 1, and either a `str` or `None` at index 2.
    * For annotations defined in this form Valimp will additionally validate that inputs are of the declared length.

In [15]:
@parse
def pf(
    a: tuple[int, ...],
    b: tuple[Union[int, float], ...],
    c: Union[tuple[int, ...], tuple[float, ...]],  # not the same as b!
    d: Union[tuple[int, ...], tuple[float, ...]],  # not the same as b!
    e: tuple[str, int, Optional[str]],
    f: tuple[str, int, Optional[str]],
    g: tuple[str, int, float, int],
    h: tuple[str, int, float, int],
):
    return

In [16]:
rtrn = pf(  # valid inputs
    a=(0, 0, 0),
    b=(1.1, 2, 2.2, 3),
    c=(2, 3, 4),
    d=(3.3, 4.4, 5.5),
    e=("four", 4, None),
    f=("five", 5, "opt_five"),
    g=("six", 6, 6.0, 6),
    h=("seven", 7, 7.0, 7),
)
assert rtrn is None

In [ ]:
pf(
    a=(0, "zero", 0),  # INVALID as includes str
    b=(1.1, "one", 2.2, 3),  # INVALID as includes str
    c=(2, 3.3, 4),  # INVALID as includes float and int (should be all float or all int)
    d=(3.3, 4.4, 5.5),  # valid
    e=(4, 4, None),  # INVALID as item at index 0 is not str
    f=("five", 5, 3),  # INVALID as item at index 2 is not str or None
    g=("six", 6, 6.0),  # INVALID as too short, should have length 4
    h=("seven", 7, 7.0, 7, 7.0),  # INVALID as too long, should have length 4
)

```
---------------------------------------------------------------------------
InputsError                               Traceback (most recent call last)
Cell In[17], line 1
----> 1 pf(

InputsError: The following inputs to 'pf' do not conform with the corresponding type annotation:

a
	Takes type <class 'tuple'> containing items that conform with <tuple[int, ...]>, although the received container contains item 'zero' of type <class 'str'>.

b
	Takes type <class 'tuple'> containing items that conform with <tuple[typing.Union[int, float], ...]>, although the received container contains item 'one' of type <class 'str'>.

c
	Takes input that conforms with <(tuple[int, ...], tuple[float, ...])> although received '(2, 3.3, 4)' of type <class 'tuple'>.

e
	Takes type <class 'tuple'> containing items that conform with <tuple[str, int, typing.Optional[str]]>, although the item in position 0 is '4' of type <class 'int'>.

f
	Takes type <class 'tuple'> containing items that conform with <tuple[str, int, typing.Optional[str]]>, although the item in position 2 is '3' of type <class 'int'>.

g
	Takes type <class 'tuple'> of length 4 although received '('six', 6, 6.0)' of length 3.

h
	Takes type <class 'tuple'> of length 4 although received '('seven', 7, 7.0, 7, 7.0)' of length 5.
```

#### `set`

`set` can be subscripted with a single argument that defines the annotation that all contained items should confirm to.

In [18]:
@parse
def pf(
    a: set[str],
    b: set[Union[int, float]],
    c: Union[set[int], set[float]],  # not the same as b!
    d: Union[set[int], set[float]],  # not the same as b!
):
    return

rtrn = pf(  # valid inputs
    a={"a", "aa", "aaa"},
    b={1, 1.1, 1.11},
    c={2, 3, 4},
    d={3.3, 4.4, 5.5},
)
assert rtrn is None

In [ ]:
pf(
    a={"a", 2, "aaa"},  # INVALID as contains an int
    b={1, 1.1, 1.11},  # valid
    c={2, 3.3, 4},  # INVALID as contains a float (should be all int or all float)
    d={3.3, 4.4, 5.5},  # valid
)

```
---------------------------------------------------------------------------
InputsError                               Traceback (most recent call last)
Cell In[19], line 1
----> 1 pf(

InputsError: The following inputs to 'pf' do not conform with the corresponding type annotation:

a
	Takes type <class 'set'> containing items that conform with <set[str]>, although the received container contains item '2' of type <class 'int'>.

c
	Takes input that conforms with <(set[int], set[float])> although received '{2, 3.3, 4}' of type <class 'set'>.
```

#### `valimp.NO_ITEM_VALIDATION`

To **not** validate a container's items just include the `valimp.NO_ITEM_VALIDATION` constant in an annotation's metadata.

Annotation metadata is defined by simply wrapping the annotation in `typing.Annotated`. The first argument of the `typing.Annotated` subscription takes the wrapped annotation, all further arguments are consumed as annotation metadata.

Notice how in the following example the parameters with `valimp.NO_ITEM_VALIDATION` are not included in the error message - they are considered valid as the container type is correct and the contained items are not validated.

In [20]:
from typing import Annotated
from valimp import NO_ITEM_VALIDATION

@parse
def pf(
    a: tuple[str, ...],
    a1: Annotated[tuple[str, ...], NO_ITEM_VALIDATION],
    b: dict[str, int],
    b1: Annotated[dict[str, int], NO_ITEM_VALIDATION],
    c: set[int],
    c1: Annotated[set[int], NO_ITEM_VALIDATION],
):
    return

In [ ]:
pf(
    a=("a", 0),  # INVALID as contains int
    a1=("a", 0),  # ...INVALID for same reason but will not raise error
    b={"bkey": "bval"},  # INVALID as has value as str
    b1={"bkey": "bval"},  # ...INVALID for same reason but will not raise error
    c={0, 1, 2.2},  # INVALID as contains a float
    c1={0, 1, 2.2},  # ...INVALID for same reason but will not raise error
)

```
---------------------------------------------------------------------------
InputsError                               Traceback (most recent call last)
Cell In[21], line 1
----> 1 pf(

InputsError: The following inputs to 'pf' do not conform with the corresponding type annotation:

a
	Takes type <class 'tuple'> containing items that conform with <tuple[str, ...]>, although the received container contains item '0' of type <class 'int'>.

b
	Takes type <class 'dict'> with values that conform to the second argument of <dict[str, int]>, although the received dictionary contains value 'bval' of type <class 'str'>.

c
	Takes type <class 'set'> containing items that conform with <set[int]>, although the received container contains item '2.2' of type <class 'float'>.
```

#### Nested containers

Valimp will by default recursively validate the types of contained items in nested containers.

In [22]:
@parse
def pf(
    a: tuple[list[set[Union[int, float]]], ...],
):
    return

rtrn = pf(
    (
        [
            {0, 0.1}, {1, 1.1}
        ],
    ),
)
assert rtrn is None

If the first nested container (the list) contains an invalid item then an error is raised (although the error message could be more insightful)...

In [ ]:
pf(
    (
        [
            {0, 0.1}, {1, 1.1},
            "not a set",
        ],
    ),
)

```
---------------------------------------------------------------------------
InputsError                               Traceback (most recent call last)
Cell In[23], line 1
----> 1 pf(

InputsError: The following inputs to 'pf' do not conform with the corresponding type annotation:

a
	Takes type <class 'tuple'> containing items that conform with <tuple[list[set[typing.Union[int, float]]], ...]>, although the received container contains item '[{0, 0.1}, {1, 1.1}, 'not a set']' of type <class 'list'>.
```

An error is also raised if the list contents are valid although one of the second level of nested containers (the sets) contains an invalid item...

In [ ]:
pf(
    (
        [
            {0, 0.1}, {1, 1.1, "one"},
        ],
    ),
)

```
---------------------------------------------------------------------------
InputsError                               Traceback (most recent call last)
Cell In[24], line 1
----> 1 pf(

InputsError: The following inputs to 'pf' do not conform with the corresponding type annotation:

a
	Takes type <class 'tuple'> containing items that conform with <tuple[list[set[typing.Union[int, float]]], ...]>, although the received container contains item '[{0, 0.1}, {1, 'one', 1.1}]' of type <class 'list'>.
```

Including `valimp.NO_ITEM_VALIDATION` to an annotation's metadata will result in the contained items not being validated at any level of nesting.

In [25]:
@parse
def pf(
    a: Annotated[tuple[list[set[Union[int, float]]], ...], NO_ITEM_VALIDATION],
) -> tuple[list[set[Union[int, float]]], ...]:
    return a

In [26]:
pf(
    (
        [
            {0, 0.1}, {1, 1.1, "one"},
            "not a set",
        ],
        "not a list",
    ),
)

([{0, 0.1}, {1, 1.1, 'one'}, 'not a set'], 'not a list')

Indeed, with `valimp.NO_ITEM_VALIDATION` as long as the input's a tuple, it'll validate...

In [27]:
pf(("I'm a tuple", 1, 2.0))

("I'm a tuple", 1, 2.0)

NB It isn't currently possible to ignore validation only from a specific level of nesting - PRs welcome!

### `collections.abc.Callable`

Valimp validates that inputs to parameters annotated with `collections.abc.Callable` are indeed callable. However, it will **not** validate any subscriptions.

Notice how the inputs in the following example do not conform with the subscriptions although the input is validated regardless. (The first argument to `collections.abc.Callable` takes a sequence of annotations describing the callable's arguments, the second argument takes the callable's return type.)

In [28]:
from collections.abc import Callable

@parse
def pf(
    a: Callable,
    b: Callable[[str], str],
    c: Callable[[str, int], str],
    d: Callable[..., str],
):
    return

def some_func(a: float, b: float) -> float:
    return a + b

# inputs are validated even though they do not conform with the subscriptions
pf(some_func, some_func, some_func, some_func)

Although an error will be raised if the input is not callable...

In [ ]:
pf(3, "can't call me", some_func, some_func)

```
---------------------------------------------------------------------------
InputsError                               Traceback (most recent call last)
Cell In[29], line 1
----> 1 pf(3, "can't call me", some_func, some_func)

InputsError: The following inputs to 'pf' do not conform with the corresponding type annotation:

a
	Takes type <class 'collections.abc.Callable'> although received '3' of type <class 'int'>.

b
	Takes type <class 'collections.abc.Callable'> although received 'can't call me' of type <class 'str'>.
```

### `typing.Literal`

By default inputs to parameters annotated with `typing.Literal` are considered valid if the input *compares as equal* to a value defined within the subscriptions.

The `typing.Literal` annotation can be useful to validate that an input is within a set of members of an `Enum`.

In [30]:
from enum import Enum
from typing import Literal

Monty = Enum('Monty', 'FOO SPAM BAR')

@parse
def pf(
    a: Literal["one", "two"],
    b: Literal[1, 2, 3],
    c: Literal[Monty.FOO, Monty.SPAM, Monty.BAR],
    d: Literal[Monty.FOO, Monty.SPAM, Monty.BAR],
    e: Literal[Monty.FOO, Monty.SPAM],
):
    return

rtrn = pf(  # valid inputs
    "one",
    2.0,  # NB not included in annotation, but valid as compares equal to 2
    Monty.BAR,
    Monty.SPAM,
    Monty.FOO,
)
assert rtrn is None

In [ ]:
pf(
    a=1,  # INVALID as does not compare equal with "one" or "two"
    b=1,  # valid
    c="BAR",  # INVALID as does not compare equal with unique enum
    d=3,  # INVALID as does not compare equal with unique enum
    e=Monty.BAR  # INVALID as member not included to annotation
)

```
---------------------------------------------------------------------------
InputsError                               Traceback (most recent call last)
Cell In[31], line 1
----> 1 pf(

InputsError: The following inputs to 'pf' do not conform with the corresponding type annotation:

a
	Takes a value from <('one', 'two')> although received '1'.

c
	Takes a value from <(<Monty.FOO: 1>, <Monty.SPAM: 2>, <Monty.BAR: 3>)> although received 'BAR'.

d
	Takes a value from <(<Monty.FOO: 1>, <Monty.SPAM: 2>, <Monty.BAR: 3>)> although received '3'.

e
	Takes a value from <(<Monty.FOO: 1>, <Monty.SPAM: 2>)> although received 'Monty.BAR'.
```

The `valimp.STRICT_LITERAL` constant can be included to the annotation metadata to only validate input if it *is* the same object as one of the objects defined within the subscriptions, i.e. it's not enough for the object to merely compare as equal, rather they have to be one and the same. (NB consider using an Enum before resorting to `valimp.STRICT_LITERAL`).

In [32]:
from valimp import STRICT_LITERAL

LIT = "spam"

@parse
def pf(
    a: Literal[LIT],
    b: Annotated[Literal[LIT], STRICT_LITERAL],
):
    return

pf(LIT, LIT)  # valid as both parameters are receiving the same object

If we define a different object which compares equal to LIT...

In [33]:
diff_obj = "spamx"[:-1]
assert diff_obj == LIT
diff_obj

'spam'

Then whilst this is valid...

In [34]:
pf(diff_obj, LIT)

This isn't (at least not for parameter b)...

In [ ]:
pf(diff_obj, diff_obj)

```
---------------------------------------------------------------------------
InputsError                               Traceback (most recent call last)
Cell In[35], line 1
----> 1 pf(diff_obj, diff_obj)

InputsError: The following inputs to 'pf' do not conform with the corresponding type annotation:

b
	Takes a literal from <('spam',)> although received 'spam'.
```

## Signature validation

Valimp also validates that inputs conform with a function's signature.

A `valimp.InputsError` will be raised if at least one of the following is true.
* A required argument is not passed (missing positional argument).
* A required keyword-only argument is not passed (missing keyword-only argument).
* A keyword argument is passed that is not represented in the signature (unexpected keyword argument).
* More arguments are passed positionally than accommodated for by the signature (excess positional arguments).
* A parameter is passed both positionally and as a keyword argument (got multiple values).

All signature errors are advised in the error message, together with any errors relating to invalid types.

In [36]:
@parse
def pf(
    a: int,
    b: int,
    *,
    kw_a: int,
):
    return

In [ ]:
pf(3, "not an int", 5, a=3, not_a_kwarg=3)

```
---------------------------------------------------------------------------
InputsError                               Traceback (most recent call last)
Cell In[37], line 1
----> 1 pf(3, "not an int", 5, a=3, not_a_kwarg=3)

InputsError: Inputs to 'pf' do not conform with the function signature:

Got multiple values for argument: 'a'.

Received 1 excess positional argument as:
	'5' of type <class 'int'>.

Got unexpected keyword argument: 'not_a_kwarg'.

Missing 1 keyword-only argument: 'kw_a'.

The following inputs to 'pf' do not conform with the corresponding type annotation:

b
	Takes type <class 'int'> although received 'not an int' of type <class 'str'>.
```

In [ ]:
pf(3, kw_a=3)

```
---------------------------------------------------------------------------
InputsError                               Traceback (most recent call last)
Cell In[38], line 1
----> 1 pf(3, kw_a=3)

InputsError: Inputs to 'pf' do not conform with the function signature:

Missing 1 positional argument: 'b'.
```

## Coerce

Valimp provides for coercing a parameter's input to a specific type.

This is useful if it's convenient for the user to pass an input as one of various types although you don't want to clutter up your function by then getting the input into the type that you'll be using internally.

An input can be coerced simply by including an instance of `valimp.Coerce` to the parameter's annotation metadata. `valimp.Coerce` takes a single argument as the class that the input is to be coerced to (under-the-bonnet the `@parse` decorator simply passes the input to the constructor of the class being coerced to).

In [39]:
from valimp import Coerce

@parse
def pf(
    a: Annotated[Union[float, int, str], Coerce(float)],
    b: Annotated[Union[float, int, str], Coerce(float)],
    c: Annotated[Union[float, int, str], Coerce(float)],
    d: Annotated[Union[float, int, str], Coerce(float)],
    e: Annotated[Union[float, int, str, None], Coerce(float)],
) -> dict[str, Optional[float]]:
    return {"a":a, "b":b, "c":c, "d":d, "e":e}

In [40]:
pf(0, 1.1, '2', '3.3', None) 

{'a': 0.0, 'b': 1.1, 'c': 2.0, 'd': 3.3, 'e': None}

Note that Valimp will not try to coerce a `None` input.

### Type checkers
If you're using a type checker it's unlikely that it'll work out that the `@parse` decorator has coerced the input to a specific type. Hence, in the above example the checker will expect that the input could still be a string and will start advising of errors when the received object is treated as a float.

In this case to 'right' the type checker it's necessary to narrow the type by including a type guard expression at the start of the function. For example in `mypy`:
```python
@parse
def f(param: typing.Annotated[Union[float, int, str], Coerce(float)):
    if typing.TYPE_CHECKING:
        assert isinstance(param, float)
```

PR's very much welcome from anyone who knows how to abstract this requirement away to within Valimp!

## Parsing

Valimp also provides for abstracting away the parsing and/or custom validation of inputs.

This is done by including an instance of `valimp.Parser` to the metadata of a parameter's annotation. The first argument of `Parser` takes a callable which should return the object as to be receieved by the decorated funcion's formal parameter. The callable should have the following signature:

```python
def parser_func(name: str, obj: Any, params: dict[str, Any]) -> Any:
```
where:
```
PARAMETERS

name :
    Name of the parameter being parsed.

obj :
    The input as passed by the client.

    `obj` will be received as coerced by any `Coerce` instance
    only if the Coerce instance is passed to typing.Annotated
    ahead of the Parser instance.

params :
    Shallow copy of earlier inputs that have already been
    validated and, if applicable, parsed and/or coerced.

RETURNS

parsed :
    Parsed, validated object.
```

So, to simply add a suffix to a str input...

In [41]:
from valimp import Parser

def add_suffix(name: str, obj: str, params: dict[str: str]) -> str:
    return obj + "_suffix"

@parse
def pf(
    a: str,
    b: Annotated[str, Parser(add_suffix)],
) -> dict[str, str]:
    return {"a":a, "b":b}

pf("input_a", "input_b")

{'a': 'input_a', 'b': 'input_b_suffix'}

The following offers a trivial example of how the parameter name and previous inputs are available to the parser.

In [42]:
from valimp import Parser

def concat_param_name(name: str, obj: str, params: dict[str: str]) -> str:
    return obj + f"_{name}"

my_parser = Parser(concat_param_name)

def concat_earlier_input(name: str, obj: str, params: dict[str: str]) -> str:
    return obj + f"_{params['a']}"

@parse
def pf(
    a: str,
    b: Annotated[str, my_parser],  # passes a reusable Parser
    c: Annotated[str, Parser(concat_earlier_input)],  
) -> dict[str, str]:
    return {"a":a, "b":b, "c":c}

pf("input_a", "input_b", "input_c")

{'a': 'input_a', 'b': 'input_b_b', 'c': 'input_c_input_a'}

Note that the input is only passed to the parser if it's type is valid. Accordingly the type of the parser's `obj` parameter can be narrowed in the knowledge that to have got this far the input type will be valided. For example, above the `obj` parameter of the parser callables can be annotated as `str` rather than `Any`.

And as invalid inputs can't reach the parser function, you won't get an error with an obtuse message raised, such as when in the above example the parser function would otherwise come across `int` + `str`.

In [ ]:
pf("valid", "valid", 3)

```
---------------------------------------------------------------------------
InputsError                               Traceback (most recent call last)
Cell In[43], line 1
----> 1 pf("valid", "valid", 3)

InputsError: The following inputs to 'pf' do not conform with the corresponding type annotation:

c
	Takes type <class 'str'> although received '3' of type <class 'int'>.
```

### Custom validation

`Parser` provides for easily abstracting away custom input validations. Just raise an error within the parser function if the input doesn't validate.

For example, to ensure that an input is greater that a specific value, or greater than the value passed for a prior parameter...

In [44]:
def simple_validator(name: str, obj: int, params: dict[str, int]) -> int:
    if obj < 10:
        raise ValueError(
           f"The value of parameter {name} cannot be less than 10."
        )
    return obj

def validator(name: str, obj: int, params: dict[str, int]) -> int:
    if obj <= params["a"]:
        raise ValueError(
            f"The value of parameter '{name}' cannot be less than"
            " the value of parameter 'a', although received 'a' as"
            f" {params['a']} and '{name}' as {obj}."
        )
    return obj

@parse
def pf(
    a: Annotated[int, Parser(simple_validator)],
    b: Annotated[int, Parser(validator)],
):
    return

assert pf(10, 15) is None

In [ ]:
pf(5, 4)

```
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Cell In[45], line 1
----> 1 pf(5, 4)

ValueError: The value of parameter a cannot be less than 10.
```

Note that in the case of custom validation the user-defined error is raised directly for the first input that fails validation, i.e. although the second input here would also fail the validation no such advices are offered to the user, to whom it probably won't become apparent until they then try...

In [ ]:
pf(10, 4)

```
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Cell In[46], line 1
----> 1 pf(10, 4)

ValueError: The value of parameter 'b' cannot be less than the value of parameter 'a', although received 'a' as 10 and 'b' as 4.
```

### Dynamic default values

Default values can be defined dynamcially with reference to earlier inputs.

For example, to simply set a parameter's default to the value of an earlier required input...

In [47]:
@parse
def pf(
    a: int,
    b: Annotated[
        Optional[int],
        Parser(lambda _, obj, params: obj if obj is not None else params["a"]),
    ] = None,
) -> tuple[int, int]:
    return a, b

In [48]:
pf(10)

(10, 10)

### parse_none

Passing the `None` value to the parser function is what provides for setting a default value. But what if `None` is a valid value in itself which you don't want to send to the parser function? This is what the `Parser` `parse_none` keyword argument is for. It's `True` by default, although if passed as `False` then a `None` input will be passed straight through to the formal parameter rather than to the parser function.

In [49]:
@parse
def pf(
    a: Annotated[
        Optional[int],
        Parser(lambda _, obj, params: obj + 5, parse_none=False),
    ] = None,
):
    return a

In [50]:
pf(3)

8

In [51]:
assert pf(None) is None

Note what would happen if `parse_none` had not been passed to `Parser` as `False`:

In [52]:
@parse
def pf(
    a: Annotated[
        Optional[int],
        Parser(lambda _, obj, params: obj + 5),
    ] = None,
):
    return a

In [ ]:
pf(None)

```
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[53], line 1
----> 1 pf(None)

File ~\valimp\src\valimp\valimp.py:931, in parse.<locals>.wrapped_f(*args, **kwargs)
    929                 if obj is None and not data.parse_none:
    930                     continue
--> 931                 obj = data.function(name, obj, new_as_kwargs.copy())
    933     new_as_kwargs[name] = obj
    935 return f(**new_as_kwargs)

Cell In[52], line 5, in <lambda>(_, obj, params)
      1 @parse
      2 def pf(
      3     a: Annotated[
      4         Optional[int],
----> 5         Parser(lambda _, obj, params: obj + 5),
      6     ] = None,
      7 ):
      8     return a

TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'
```

## Coerce and parse

Instances of `valimp.Coerce` and `valimp.Parser` can both be passed to the same type annotation.

In this case, order matters. If the `Coerce` instance is passed ahead of the `Parser` instance then the parser function will receive the input post-coercion. If however the `Parser` instance is passed ahead of the `Coerce` instance then the parser function will receive the user's input directly and the parser's output will be subsequently coerced.

The following offers an example of the more common case of coercing then validating an input.

In [54]:
def _validate(name: str, obj: float, params: dict[str: Any]) -> float:
    if obj < params["a"]:
        raise ValueError(
            f"If passed then the value of parameter '{name}' cannot be less"
            f" than the value of parameter 'a', although received 'a' as"
            f" {params['a']} and '{name}' as {obj}."
        )
    return obj    

validate = Parser(_validate, parse_none=False)

@parse
def pf(
    a: Annotated[Union[float, int, str], Coerce(float)],
    b: Annotated[
        Union[float, int, str, None],
        Coerce(float),
        validate,
    ] = None,
) -> tuple[float, Optional[float]]:
    return a, b

In [55]:
pf("2.2")

(2.2, None)

In [56]:
pf("2.2", "3.3")

(2.2, 3.3)

In [57]:
pf("2.2", 3)

(2.2, 3.0)

In [ ]:
pf("2.2", "1.8")

```
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Cell In[58], line 1
----> 1 pf("2.2", "1.8")

File ~\valimp\src\valimp\valimp.py:931, in parse.<locals>.wrapped_f(*args, **kwargs)
    929                 if obj is None and not data.parse_none:
    930                     continue
--> 931                 obj = data.function(name, obj, new_as_kwargs.copy())
    933     new_as_kwargs[name] = obj
    935 return f(**new_as_kwargs)

Cell In[54], line 3, in _validate(name, obj, params)
      1 def _validate(name: str, obj: float, params: dict[str: Any]) -> float:
      2     if obj < params["a"]:
----> 3         raise ValueError(
      4             f"If passed then the value of parameter '{name}' cannot be less"
      5             f" than the value of parameter 'a', although received 'a' as"
      6             f" {params['a']} and '{name}' as {obj}."
      7         )
      8     return obj

ValueError: If passed then the value of parameter 'b' cannot be less than the value of parameter 'a', although received 'a' as 2.2 and 'b' as 1.8.
```